In [1]:
from transformers import pipeline

显存优化：
优化一:将batch_size设为1，（但是原来的batch_size = 32,为了不影响效果，将梯度累加设置成32个batch算一次梯度）
    梯度累加修改的地方：train_args的参数：gradient_accumulation_step = 32
优化二：将gradient_checkpointing改为true
-修改后的前向传播: 在前向传播时，它不再保存所有的中间激活值，只保存少数几个关键的（即 Checkpoints）。大部分激活值用完即弃，不占用显存。
-修改后的反向传播: 在反向传播需要某个被丢弃的激活值时，它会找到最近的一个 Checkpoint，然后重新执行一小段前向传播计算，来“恢复”那个需要的激活值。
优点：显存占用减少。缺点，训练时间增加30%
优化三：使用现存占用更小的优化器，如将adam改为adamfactor
优化四：冻结bert的参数，只训练全连接层的参数：遍历所有的参数，将param.requires_grad = false
for name,param in model.bert.name_parameters()
    param.require_grad = False
优化五：在tokenzizer里将maxlengh改小